In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1665543743940_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-2>

In [4]:
sc.install_pypi_package('boto3')
import boto3
comprehend= boto3.client('comprehend', region_name='us-east-1')
print(comprehend.detect_sentiment(Text='Blackpink in your area', LanguageCode='en'))
comprehend.close()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'Sentiment': 'NEUTRAL', 'SentimentScore': {'Positive': 0.3396611511707306, 'Negative': 0.15249323844909668, 'Neutral': 0.43417444825172424, 'Mixed': 0.0736711174249649}, 'ResponseMetadata': {'RequestId': 'db249b21-0f04-4ef4-a676-3ac4442333a3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'db249b21-0f04-4ef4-a676-3ac4442333a3', 'content-type': 'application/x-amz-json-1.1', 'content-length': '160', 'date': 'Wed, 12 Oct 2022 04:25:41 GMT'}, 'RetryAttempts': 0}}

### Load business.json and review.json from S3

In [12]:
df_business=spark.read.format('json').load('s3://ahd6901-yelp-bucket/yelp_academic_dataset_business.json')
df_review= spark.read.format('json').load('s3://ahd6901-yelp-bucket/yelp_academic_dataset_review.json')
print(df_business.columns)
df_business.show(10)
print(df_review.columns)
df_review.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['attributes', 'business_id', 'categories', 'city', 'full_address', 'hours', 'latitude', 'longitude', 'name', 'neighborhoods', 'open', 'review_count', 'stars', 'state', 'type']
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|          attributes|         business_id|          categories|      city|        full_address|               hours|  latitude|  longitude|                name|neighborhoods| open|review_count|stars|state|    type|
+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----------+-----------+--------------------+-------------+-----+------------+-----+-----+--------+
|[,,,,,,,, true,,,...|vcNAWiLM4dR7D2nww...|[Doctors, Health ...|   Phoenix|4840 E Indian Sch...|[[17:00, 08:00], ...| 33.499313|-111.983758|   Eric Goldberg, MD|           []|

### Check for duplicates and remove if they exist. Display needed columns

In [13]:
if df_review.count() > df_review.dropDuplicates(['text']).count():
    df_review=df_review.dropDuplicates(['text'])
df_business[['business_id', 'name', 'categories']].show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|         business_id|                name|          categories|
+--------------------+--------------------+--------------------+
|vcNAWiLM4dR7D2nww...|   Eric Goldberg, MD|[Doctors, Health ...|
|JwUE5GmEO-sH1FuwJ...|Pine Cone Restaurant|       [Restaurants]|
|uGykseHzyS5xAMWoN...|Deforest Family R...|[American (Tradit...|
|LRKJF43s9-3jG9Lgx...|            Culver's|[Food, Ice Cream ...|
|RgDg-k9S5YD_BaxMc...|Chang Jiang Chine...|[Chinese, Restaur...|
|oLctHIA1AxmsgOuu4...|Charter Communica...|[Television Stati...|
|ZW2WeP2Hp20tq0RG1...| Air Quality Systems|[Home Services, H...|
|95p9Xg358BezJyk1w...|McFarland Public ...|[Libraries, Publi...|
|rdAdANPNOcvUtoFgc...|Green Lantern Res...|[American (Tradit...|
|ZlhAvs_6ryi4_efZ1...|Spartan Animal Ho...|[Veterinarians, P...|
|_wZTYYL7cutanzAnJ...|Beach House Resta...|[Bars, American (...|
|XrzTVrJAjs0yYLvhF...|Parsonage Bed & B...|[Hotels & Travel,...|
|1tkeiIa-daD8LbM6m...|   

### Find businesses that only serve either Vietnamese, Chinese, or Thai cuisine

In [14]:
def get_fusion_business(cuisine_a, cuisine_b, cusine_exclude):
    return df_business.filter(fn.array_contains(df_business.categories, cuisine_a)
                              & fn.array_contains(df_business.categories, cuisine_b)
                              & ~fn.array_contains(df_business.categories, cuisine_exclude)).select('business_id', 'name').distinct()
def get_business_df(main_cuisine,b,c):
    return df_business.filter(fn.array_contains(df_business.categories, main_cuisine) 
                              & ~fn.array_contains(df_business.categories, b)
                              & ~fn.array_contains(df_business.categories, c)).select('business_id','name').distinct()
df_viet_business= get_business_df('Vietnamese','Chinese','Thai')
df_chinese_business=get_business_df('Chinese','Vietnamese','Thai')
df_thai_business=get_business_df('Thai','Vietnamese','Chinese')
print('#Businesses serve only Vietnamese cusine: ', df_viet_business.count())
print('#Businesses serve only Chinese cusine: ', df_chinese_business.count())
print('#Businesses serve only Thai cusine: ', df_thai_business.count())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#Businesses serve only Vietnamese cusine:  130
#Businesses serve only Chinese cusine:  950
#Businesses serve only Thai cusine:  258

### Find each cuisine reviews

In [21]:
def get_reviews(b_df):
    return b_df.join(df_review,df_review.business_id ==  b_df.business_id,"inner").select(df_review.business_id, b_df.name, df_review.text)
viet_cuisine_reviews=get_reviews(df_viet_business)
chinese_cuisine_reviews=get_reviews(df_chinese_business)
chinese_thai_reviews=get_reviews(df_thai_business)
viet_cuisine_reviews.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|         business_id|                name|                text|
+--------------------+--------------------+--------------------+
|3OauvrqUU8nDLWcW6...|        Viet Kitchen|"Buyer beware"  
...|
|r0ZoytDjK4CY5ytFN...|             Kinh Do|$3 Banh Mi beats ...|
|fCkgtZUUM5qmU_-KW...| Pho Hung Restaurant|....... ? this is...|
|CMTDZRDnv_O0rwAvR...|BlueMoon Vietname...|1) Decent tasting...|
|tqDwpyCB53TiEIv91...|           Pho Avina|3.5 *
The lady wh...|
|jtzhY-P4H6WSYpv5r...|          Rice Paper|90-ish year old s...|
|lHRc-t5EmkY9ejWix...|         Viet Bistro|A few years ago, ...|
|lHRc-t5EmkY9ejWix...|         Viet Bistro|After enjoying th...|
|Zq-yfLJYy7245p-O8...|         Viet Bistro|After reading the...|
|17FKJgS11HppOf00q...|     Lemongrass Café|After searching o...|
+--------------------+--------------------+--------------------+
only showing top 10 rows

### Detect each review's sentiment

In [24]:
def get_sentiment(text):
    comprehend=boto3.client('comprehend', region_name='us-east-1')
    res=comprehend.detect_sentiment(Text=text, LanguageCode='en')
    comprehend.close()
    return res['Sentiment']
def get_sentiment_df(review_df):
    sentiment_rdd= review_df.rdd.map(lambda x:(x[2], get_sentiment(x[2])))
    return sentiment_rdd.toDF(['text', 'sentiment'])

viet_cusine_sentiment= get_sentiment_df(viet_cuisine_reviews)
viet_cusine_sentiment.show(15)
chinese_cusine_sentiment= get_sentiment_df(chinese_cuisine_reviews)
thai_cusine_sentiment= get_sentiment_df(chinese_thai_reviews)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|"Buyer beware"  
...| NEGATIVE|
|$3 Banh Mi beats ...| POSITIVE|
|....... ? this is...| POSITIVE|
|1) Decent tasting...|    MIXED|
|3.5 *
The lady wh...| POSITIVE|
|90-ish year old s...| POSITIVE|
|A few years ago, ...| POSITIVE|
|After enjoying th...| POSITIVE|
|After reading the...|    MIXED|
|After searching o...| NEGATIVE|
|All I can say is ...| POSITIVE|
|All I could say i...| POSITIVE|
|All of the Vietna...| POSITIVE|
|Although this Vie...| POSITIVE|
|Am I allowed to j...| NEGATIVE|
+--------------------+---------+
only showing top 15 rows

### Load results to a new DataFrame and save it as a CSV file in S3 bucket 

In [27]:
columns=['Cuisine', 'mixed_review_count', 'neutral_review_count', 'negative_review_count', 'positive_review_count', 'total_review_count']
data=[]
def create_row(cuisine_name, df_sentiment):
    row=[cuisine_name]
    data_collect= df_sentiment.groupBy('sentiment').count().collect()
    for r in data_collect:
        row.append(r['count'])
    row.append(df_sentiment.count())
    return row
data.append(create_row('Vietnamese', viet_cusine_sentiment))
data.append(create_row('Chinese', chinese_cusine_sentiment))
data.append(create_row('Thai', thai_cusine_sentiment))
rdd= spark.sparkContext.parallelize(data)
result_df= rdd.toDF(columns)
result_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+--------------------+---------------------+---------------------+------------------+
|   Cuisine|mixed_review_count|neutral_review_count|negative_review_count|positive_review_count|total_review_count|
+----------+------------------+--------------------+---------------------+---------------------+------------------+
|Vietnamese|              1821|                  68|                 1432|                 6134|              9454|
|   Chinese|              6876|                 272|                 6504|                20396|             34045|
|      Thai|              3063|                  90|                 2041|                11894|             17090|
+----------+------------------+--------------------+---------------------+---------------------+------------------+

In [28]:
result_df.coalesce(1)\
                    .write.format('csv').option("header", "true")\
                    . save("s3://ahd6901-yelp-bucket/result2/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…